## Gather Follower Descriptions
We start with a twitter user name (as well as things like twitter credentials) and pull all the followers of that user. We then pull the descriptions of those users and write them to a file for future mining. You can use more than one user in a group.

I wrote some functions for you in the file `twitter_functions.py`. This will need to be in the same folder where you're running this notebook. 

You may not have installed `tweepy` yet. If you get an error on the next cell, you need to go to the command/console line (like where you type `git` commands) and type something like `pip install tweepy`. There may be dependencies (like `unidecode`) that you need as well. 

In [ ]:
from datetime import datetime
import json
from pprint import pprint

from twitter_functions import * # these are the functions I wrote for you.

The next cell holds your Twitter authorization credentials. Then it calls a function that initializes your connection to Twitter. I've left my keys in there (slightly perturbed so that they won't actually work) so that you see the form these take).

In [ ]:
#from twitter_config import *

auth =  { "consumer_key": "xks2XTK4gr2PajPio1RBGWsYU",
          "consumer_secret": "jkjkCjph2vx38uBVbVHLkhzesGVY6ZqEywXd3B0sDeSAWVcDNo",
          "access_key": "33029025-i1Mm907o7BsKnufMIxjVzByKsuDEhOBb0yV3EAa1E",
          "access_secret": "jkjkXqAijIwRQMZmW3b7AgpFXU6Ve0RU30fzsbzpfx9uf"
        }

api = initialize_twitter(auth)

Now you set the handle (or handles) that represent one group or topic on Twitter. These should be in a list. The output file name (`ofile_name`) is determined based on today's date and the first element in the list. Feel free to modify. 

In [ ]:
#starting_user = ['GeneralMills','KraftBrand'] # my first group
starting_user = ['michaelpollan'] # my second group

ofile_name = (datetime.today().strftime("%Y%m%d") + "_" + 
             starting_user[0] + "_" + # Just use the first starting_user for the name if there are multiple
             "followers.txt")

In [ ]:
# We'll now go lookup the full information on your starting user(s). 
starting_user_id = []

# All records will be a dictionary with the twitter ID as the key and 
# a UserRecord as the value. This is a named tuple I created. 
all_records = lookup_users_from_handles(api, starting_user)

# We need the IDs that we're getting followers from in a list. 
for id in all_records : #access the keys, which are ids.
    starting_user_id.append(id)

In [ ]:
all_records

In [ ]:
# How long is it going to take us to pull these followers?
total_followers = 0
for id, rec in all_records.items() :
    total_followers += rec.followers_count
    
print("Ooh, {fol:,d} followers. A complete run with no limits run is ".format(fol=total_followers) + 
      "going to take {min:.2f} minutes ({hour:.2f} hours)".format(min=total_followers/5000,
                                                                  hour=total_followers/(60*5000)))

In [ ]:
# Now let's pull all the followers of our starting_user
# the function I wrote allows you to cap the number of followers you pull
# and uses the ID to generate the query.
# 
# Note that this pull is subject to rate limiting. You can make 15 calls per
# 15 minute window and each can return 5000 users. 
followers_of_starting = gather_followers(api,
                                         starting_user_id,
                                         follower_limit=5000) # Modify this limit if you need to. 
                                                              # Set it to "None" to get all   

# followers_of_starting will be a dictionary with the key being the id(s) in starting_user_id
# and the value is a list of all the followers.

In [ ]:
# And now we'll "hydrate" these user records.
for start_id, list_of_followers in followers_of_starting.items() :
    
    # Using a set here instead of a list so that we pull each ID only once.
    ids_to_hydrate = {id for id in list_of_followers if id not in all_records} 
    
    these_records = lookup_users_from_ids(api,ids=ids_to_hydrate)

    for id, rec in these_records.items() :
        all_records[id] = rec


In [ ]:
# Now let's write out all the records. I wrote some functions to help.
with open(ofile_name,'w') as ofile :
    write_user_rec_headers(ofile)
    for id, rec in all_records.items() :
        write_user_rec(ofile, rec)